# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846853755446                   -0.70    4.8         
  2   -7.852325475257       -2.26       -1.53    1.0   24.4ms
  3   -7.852615475817       -3.54       -2.55    1.5   26.4ms
  4   -7.852645978387       -4.52       -2.89    2.8   33.8ms
  5   -7.852646521678       -6.26       -3.21    1.0   24.7ms
  6   -7.852646678755       -6.80       -3.99    1.0   24.8ms
  7   -7.852646686311       -8.12       -5.26    1.5   28.1ms
  8   -7.852646686724       -9.38       -5.46    2.5   35.2ms
  9   -7.852646686730      -11.28       -6.62    1.0   25.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846801294325                   -0.70           4.8         
  2   -7.852555105665       -2.24       -1.63   0.80    2.0    322ms
  3   -7.852639472023       -4.07       -2.73   0.80    1.0   22.7ms
  4   -7.852646512646       -5.15       -3.39   0.80    2.0   27.6ms
  5   -7.852646680035       -6.78       -4.29   0.80    1.5   24.7ms
  6   -7.852646686621       -8.18       -4.82   0.80    2.0   28.4ms
  7   -7.852646686720      -10.00       -5.67   0.80    1.2   24.3ms
  8   -7.852646686730      -11.02       -6.32   0.80    2.2   28.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.397936e+01     3.671649e+00
 * time: 0.4098169803619385
     1     1.413390e+00     1.921467e+00
 * time: 0.6038758754730225
     2    -1.255853e+00     2.371135e+00
 * time: 0.6287798881530762
     3    -3.672617e+00     1.991315e+00
 * time: 0.6644330024719238
     4    -5.003933e+00     1.830372e+00
 * time: 0.6999249458312988
     5    -6.822543e+00     1.041037e+00
 * time: 0.7352800369262695
     6    -7.455369e+00     4.707801e-01
 * time: 0.7708508968353271
     7    -7.703309e+00     2.479563e-01
 * time: 0.7954459190368652
     8    -7.788945e+00     1.046308e-01
 * time: 0.8197710514068604
     9    -7.812252e+00     1.563123e-01
 * time: 0.8443939685821533
    10    -7.832874e+00     4.620092e-02
 * time: 0.8688030242919922
    11    -7.843145e+00     5.613406e-02
 * time: 0.8931820392608643
    12    -7.846602e+00     5.429516e-02
 * time: 0.9176840782165527
    13    -7.846986e+00     3.732356e-02
 * time: 0.9421689510

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846792963934                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645892005                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.50s
  3   -7.852646686730      -13.24       -7.23    168ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.0086462569240137e-7
|ρ_newton - ρ_scfv| = 8.383718947048147e-8
|ρ_newton - ρ_dm|   = 5.853950607452973e-10
